Ho 3 cell-lines, per 2 mi danno tutto e mi hanno anche separato in train and validation, per il terzo mi danno solo le x
In cage_train mi danno le x che sono dove sono i picchi di produzione per ogni gene
Poi mi danno informazioni aggiuntive in DNAsi (quanto è accessibile il DNA), e marker degli histoni (Quelli che incominciano con la H..)
La cosa importante per il risultato e che i numeri sono nell ordine giusto, non importa la magnitudo di questi numeri
Per calcolare l errore usano spearman, che controlla solo in rank

Quello che alleno sul chromosoma 1 va bene anche sul 3, oppure anche se avessero lo stesso code, non sono comparabili

In [7]:
# Import libraries that are required to run your project
# You are allowed to add more libraries as you need

import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import torch
import torch.nn as nn

## Work Package 1.1 - Modeling Choices & Data Pre-processing

In [3]:
def process_dataframe(df_x, df_y):
    # Merge x and y on gene_name
    df = pd.merge(df_x, df_y, on='gene_name')

    # Invert given the strand
    mask = df['strand'] == '-'
    df.loc[mask, ['TSS_start', 'TSS_end']] = df.loc[mask, ['TSS_end', 'TSS_start']].values
    df = df.drop(columns=['strand'])

    # Replace chromosome with numeric values
    def chr_to_num(chr_val):
        chr_val = chr_val.replace('chr','')
        return int(chr_val)
    df['chr'] = df['chr'].apply(chr_to_num)
    
    # Sort by chromosome
    df = df.sort_values(by='chr')

    # Split into list of DataFrames by chromosome
    xs_list = [group for _, group in df.groupby('chr')]

    gene_lists = []
    ys_list = []
    xs_numpy = []
    ys_numpy = []

    for i, df_chr in enumerate(xs_list):
        gene_list = df_chr.pop('gene_name').tolist()  # extract gene_name
        y = df_chr.pop('gex').tolist()                # extract target values
        df_chr = df_chr.drop(columns=['chr'])         # remove chr column
        xs_list[i] = df_chr                            # update the list
        gene_lists.append(gene_list)                  # store gene names
        ys_list.append(y)                              # store target values

        # Convert DataFrame to numpy array
        xs_numpy.append(df_chr.to_numpy())
        # Convert target list to numpy array
        ys_numpy.append(np.array(y))

    return xs_numpy, ys_numpy, gene_lists

In [ ]:
#Cage data
cage_path = r'Data/CAGE-train/'  # raw string or forward slashes

train_x1 = pd.read_csv(cage_path + 'X1_train_info.tsv', sep='\t', usecols=[0,1,4,5,6])
train_y1 = pd.read_csv(cage_path + 'X1_train_y.tsv', sep='\t')
valid_x1 = pd.read_csv(cage_path + 'X1_val_info.tsv', sep='\t', usecols=[0,1,4,5,6])
valid_y1 = pd.read_csv(cage_path + 'X1_val_y.tsv', sep='\t')
train_x2 = pd.read_csv(cage_path + 'X2_train_info.tsv', sep='\t', usecols=[0,1,4,5,6])
train_y2 = pd.read_csv(cage_path + 'X2_train_y.tsv', sep='\t')
valid_x2 = pd.read_csv(cage_path + 'X2_val_info.tsv', sep='\t', usecols=[0,1,4,5,6])
valid_y2 = pd.read_csv(cage_path + 'X2_val_y.tsv', sep='\t')
test_x = pd.read_csv(cage_path + 'X3_test_info.tsv', sep='\t', usecols=[0,1,4,5,6])

t_x1, t_y1, t_names1 = process_dataframe(train_x1, train_y1)
t_x2, t_y2, t_names2 = process_dataframe(train_x2, train_y2)

train_x = t_x1 + t_x2
train_y = t_y1 + t_y2
train_names = t_names1 + t_names2

v_x1, v_y1, v_names1 = process_dataframe(valid_x1, valid_y1)
v_x2, v_y2, v_names2 = process_dataframe(valid_x2, valid_y2)

valid_x = v_x1 + v_x2
valid_y = v_y1 + v_y2
valid_names = v_names1 + v_names2


In [ ]:
# Dnase data


## Work Package 1.2 - Model Building

In [ ]:
class GeneModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(2, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.Conv1d(32, 16, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.Conv1d(16, 1, kernel_size=1)  # output one number per gene
        )

    def forward(self, x):
        # x shape: (batch, features=2, genes)
        out = self.net(x)
        return out.squeeze(1)  # shape: (batch, genes)

# Example usage:
model = GeneModel()


torch.Size([1, 100])


In [ ]:
##TRAINING AND EVALUATION LOGIC GOES HERE##


## Work Package 1.3 - Prediction on Test Data (Evaluation Metric)

In [ ]:
# ---------------------------INSERT CODE HERE---------------------------




# ----------------------------------------------------------------------

# Check if "pred" meets the specified constrains
assert isinstance(pred, np.ndarray), 'Prediction array must be a numpy array'
assert np.issubdtype(pred.dtype, np.number), 'Prediction array must be numeric'
assert pred.shape[0] == len(test_genes), 'Each gene should have a unique predicted expression'

In [14]:
model.eval()  # Set the model to evaluation mode
preds = []
for X_chr, y_chr in zip(train_x, train_y):
    # Convert numpy arrays to torch tensors
    X_chr = torch.tensor(X_chr.T, dtype=torch.float32).unsqueeze(0)  # (1, 2, num_genes)
    y_chr = torch.tensor(y_chr, dtype=torch.float32).unsqueeze(0)    # (1, num_genes)

    preds.append(model(X_chr))


In [21]:
# Convert to numpy if tensor
pred_values = preds[0].detach().cpu().numpy().ravel()  # assuming same chromosome index
gene_names_chr = train_names[0]  # assuming same chromosome index

# Build DataFrame
df_pred = pd.DataFrame({
    'gene_name': gene_names_chr,
    'prediction': pred_values
})

print(df_pred)

     gene_name   prediction
0       PRSS56  1251079.250
1        MZT2B  2166804.250
2         POMC -7441639.000
3      TMEM177  2736535.750
4        TTC7A -3640067.500
...        ...          ...
1198     CASP8 -3566437.750
1199     TRPM8  3872184.750
1200   FAM166C -8041816.000
1201   DNAJC10  1045738.125
1202   SLC20A1  2441869.500

[1203 rows x 2 columns]


#### Store Predictions in the Required Format

In [ ]:
# Store predictions in a ZIP. 
# Upload this zip on the project website under "Your submission".
# Zip this notebook along with the conda environment (and README, optional) and upload this under "Your code".

save_dir = 'path/to/save/output/file'  # TODO
file_name = 'gex_predicted.csv'         # PLEASE DO NOT CHANGE THIS
zip_name = "LastName_FirstName_Project1.zip" # TODO
save_path = f'{save_dir}/{zip_name}'
compression_options = dict(method="zip", archive_name=file_name)

test_genes['gex_predicted'] = pred.tolist()
test_genes[['gene_name', 'gex_predicted']].to_csv(save_path, compression=compression_options)